In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808326 sha256=7f2ca91bf49754bb80170414cbb6b2863edbd3b5c9f36a83bda87936099badf7
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import glob
from datetime import timedelta
from lightfm.cross_validation import random_train_test_split

from lightfm import LightFM
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from lightfm.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Capstone/'

Mounted at /content/drive


In [4]:
df = pd.read_parquet(path + 'Data/combined_data.parquet')

,Fund,Ticker,Backtest Date,Release Date,Data Date,Ratios Date,Price to Earnings Ratio (ttm),Price to Sales Ratio (ttm),EV/EBITDA,EV/Sales,...,Return On Invested Capital,Dividend Payout Ratio,Net Debt / EBITDA,date_entry,date_exit_1M,date_exit_3M,date_exit_6M,date_exit_12M,Type,is_holding
32,MPGEX,A,2020-09-30,2020-07-15,2020-09-30,2020-09-01,43.914828,5.755935,26.739885,5.945100,...,0.11345,0.31548,0.81857,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,1
33,MPGEX,AA,2020-09-30,2020-07-15,2020-09-30,2020-07-29,-3.726207,0.261709,6.734755,0.517962,...,-0.05596,0.00000,1.46196,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
35,MPGEX,AAL,2020-09-30,2020-07-15,2020-09-30,2020-07-23,-3.001139,0.154537,35.769533,1.034463,...,-0.06796,-0.03776,0.00000,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
37,MPGEX,AAOI,2020-09-30,2020-07-15,2020-09-30,2020-08-06,-3.212451,1.287058,-14.690248,1.949854,...,-0.19874,0.00000,-4.23090,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
39,MPGEX,AAP,2020-09-30,2020-07-15,2020-09-30,2020-08-18,25.342499,1.111888,15.171548,1.336143,...,0.06623,0.09528,2.40236,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785422,VMGRX,ZOM,2024-06-30,2024-06-14,2024-06-30,2024-05-09,-5.010351,7.110966,-3.294843,3.061792,...,-0.03906,0.00000,23.33475,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785424,VMGRX,ZS,2024-06-30,2024-06-14,2024-06-30,2024-06-07,-229.809707,13.566045,-417.228606,13.186697,...,-0.02711,0.00000,-26.06511,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785426,VMGRX,ZTS,2024-06-30,2024-06-14,2024-06-30,2024-05-02,32.217037,8.826656,21.733048,9.286592,...,0.24397,0.29946,1.23835,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785428,VMGRX,ZUMZ,2024-06-30,2024-06-14,2024-06-30,2024-06-10,-5.384205,0.382915,-9.598622,0.457494,...,-0.10932,0.00000,-1.66775,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0


In [5]:
df['Backtest Date'] = pd.to_datetime(df['Backtest Date'])
date_unique_count = df['Backtest Date'].nunique()
date = df['Backtest Date'].unique()
symbol_mf_unique_count = df['Fund'].nunique()
symbol_mf=df['Fund'].unique()
print("Unique date count:", date_unique_count)
print("Unique fund count:", symbol_mf_unique_count)


Unique date count: 13
Unique fund count: 382


# Get company scores

In [118]:
def generate_company_scores(df_selected_date):
    scores_dict = {}
    company_scores = {}
    for symbol, group in df_selected_date.groupby('Fund'):
        print(symbol)
        df_temp=group.drop(columns=['Backtest Date', 'Release Date', 'Data Date','Ratios Date',  'date_entry','date_exit_1M', 'date_exit_3M', 'date_exit_6M', 'date_exit_12M', 'Type'])
        df_temp.fillna(0, inplace=True)
        scores_dict[symbol] = generate_50_company_list(df_temp,symbol)

    for symbol, company_scores_list in scores_dict.items():

        top_companies = company_scores_list[:50]
        for index, row in company_scores_list.iterrows():
          company = row['symbol']
          score = row['score']
          # print(f'score: {score}')
          if company in company_scores:
            company_scores[company] += score
          else:
            company_scores[company] = score
    company_scores = pd.DataFrame(list(company_scores.items()), columns=['company', 'score'])
    return company_scores

In [119]:
def generate_50_company_list(df_temp,symbol):
    train_df, test_df = train_test_split(df_temp, test_size=0.2, random_state=42)
    dataset = Dataset()
    dataset.fit(
        users=df_temp['Fund'].unique(),
        items=df_temp['Ticker'].unique(),
        item_features=df_temp.columns.difference(['Ticker', 'Fund', 'is_holding']))
    (interactions_train, _) = dataset.build_interactions(
        ((row['Fund'], row['Ticker'], row['is_holding']) for _, row in train_df.iterrows()))
    item_features_train = dataset.build_item_features(
        (row['Ticker'], row[train_df.columns.difference(['Ticker', 'Fund', 'is_holding'])].to_dict()) for _, row in train_df.iterrows())

    (interactions_test, _) = dataset.build_interactions(
        ((row['Fund'], row['Ticker'], row['is_holding']) for _, row in test_df.iterrows()))
    model = LightFM(loss='warp', random_state=42)
    model.fit(interactions_train, item_features=item_features_train, epochs=10)
    user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
    if symbol in user_id_map:
        fund_idx = user_id_map[symbol]
        all_item_ids = np.array(list(item_id_map.values()))
        scores = model.predict(fund_idx, all_item_ids, item_features=item_features_train)
        scaler = MinMaxScaler()
        scores_normalized = scaler.fit_transform(scores.reshape(-1, 1)).flatten()

        stock_symbols = np.array(list(item_id_map.keys()))
        stock_scores = pd.DataFrame({'symbol': stock_symbols, 'score': scores_normalized})

        stock_scores = stock_scores.sort_values(by='score', ascending=False)
        return stock_scores
    else:
        print(f"FUND '{symbol}' not existing")
        return None


In [120]:
company_scores = pd.read_csv(path + 'Data/company_scores.csv')
company_scores = company_scores[company_scores.columns.drop('Unnamed: 3')]
company_scores

,company,score,Date
0,BEAM,3.956779,2020/9/30
1,RVNC,3.941453,2020/9/30
2,MRSN,3.933637,2020/9/30
3,SPCE,3.927296,2020/9/30
4,NKTX,3.923387,2020/9/30
...,...,...,...
29526,BRZE,82.028610,2024/6/30
29527,ATR,75.478699,2024/6/30
29528,YOU,66.727634,2024/6/30
29529,NLY,58.812526,2024/6/30


In [121]:
combined_data = pd.read_parquet(path + 'Data/combined_data.parquet')
combined_data

,Fund,Ticker,Backtest Date,Release Date,Data Date,Ratios Date,Price to Earnings Ratio (ttm),Price to Sales Ratio (ttm),EV/EBITDA,EV/Sales,...,Return On Invested Capital,Dividend Payout Ratio,Net Debt / EBITDA,date_entry,date_exit_1M,date_exit_3M,date_exit_6M,date_exit_12M,Type,is_holding
32,MPGEX,A,2020-09-30,2020-07-15,2020-09-30,2020-09-01,43.914828,5.755935,26.739885,5.945100,...,0.11345,0.31548,0.81857,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,1
33,MPGEX,AA,2020-09-30,2020-07-15,2020-09-30,2020-07-29,-3.726207,0.261709,6.734755,0.517962,...,-0.05596,0.00000,1.46196,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
35,MPGEX,AAL,2020-09-30,2020-07-15,2020-09-30,2020-07-23,-3.001139,0.154537,35.769533,1.034463,...,-0.06796,-0.03776,0.00000,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
37,MPGEX,AAOI,2020-09-30,2020-07-15,2020-09-30,2020-08-06,-3.212451,1.287058,-14.690248,1.949854,...,-0.19874,0.00000,-4.23090,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
39,MPGEX,AAP,2020-09-30,2020-07-15,2020-09-30,2020-08-18,25.342499,1.111888,15.171548,1.336143,...,0.06623,0.09528,2.40236,2020-09-30,2020-10-30,2020-12-30,2021-03-30,2021-09-30,growth,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785422,VMGRX,ZOM,2024-06-30,2024-06-14,2024-06-30,2024-05-09,-5.010351,7.110966,-3.294843,3.061792,...,-0.03906,0.00000,23.33475,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785424,VMGRX,ZS,2024-06-30,2024-06-14,2024-06-30,2024-06-07,-229.809707,13.566045,-417.228606,13.186697,...,-0.02711,0.00000,-26.06511,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785426,VMGRX,ZTS,2024-06-30,2024-06-14,2024-06-30,2024-05-02,32.217037,8.826656,21.733048,9.286592,...,0.24397,0.29946,1.23835,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0
1785428,VMGRX,ZUMZ,2024-06-30,2024-06-14,2024-06-30,2024-06-10,-5.384205,0.382915,-9.598622,0.457494,...,-0.10932,0.00000,-1.66775,2024-06-30,2024-07-30,2024-09-30,2024-12-30,2025-06-30,growth,0


In [122]:
drop_cols = [
            'Backtest Date', 'Release Date', 'Data Date','Ratios Date',
            'date_entry','date_exit_1M', 'date_exit_3M', 'date_exit_6M',
            'date_exit_12M', 'Type', 'is_holding', 'Fund', 'Ticker'
           ]
feature_cols = combined_data.columns.difference(drop_cols)
feature_cols

Index(['Debt Ratio', 'Dividend Payout Ratio', 'Dividend Yield', 'EV/EBITDA',
       'EV/FCF', 'EV/Sales', 'Net Debt / EBITDA', 'Operating Margin',
       'Price to Earnings Ratio (ttm)', 'Price to Sales Ratio (ttm)',
       'Return On Invested Capital', 'Return on Assets', 'Return on Equity'],
      dtype='object')

In [140]:
from scipy.stats import mannwhitneyu

def find_top_ratios(df, top_n, feature_cols):

    non_holdings = df[df['is_holding'] == 0]
    holdings = df[df['is_holding'] == 1]

    non_holding = non_holdings[feature_cols]
    holding = holdings[feature_cols]

    p_values = {}

    for column in non_holding.columns:
        holding_data = holding[column].dropna()
        non_holding_data = non_holding[column].dropna()

        if holding_data.size > 0 and non_holding_data.size > 0:
            u_stat, p_value = mannwhitneyu(holding_data, non_holding_data, alternative='two-sided')
            p_values[column] = p_value
        else:
          print(f'no holding history')
          continue

        # u_stat, p_value = mannwhitneyu(holding[column].dropna(), non_holding[column].dropna(), alternative='two-sided')
        # p_values[column] = p_value

    rank = pd.DataFrame(list(p_values.items()), columns=['ratio', 'p_value'])
    rank = rank.sort_values(by='p_value')
    rank['rank'] = rank['p_value'].rank(method='min')
    rank.set_index('ratio', inplace=True)

    top_ratios = rank.index[:top_n].tolist()

    return top_ratios

In [124]:
def generate_scores(df, ratios, score_set, feature_cols):
    X = df[feature_cols]
    X = X.apply(pd.to_numeric, errors='coerce')
    X.fillna(0, inplace=True)

    y = df['score']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.coef_})
    feature_importance['abs_importance'] = feature_importance['importance'].abs()
    feature_importance_sorted = feature_importance.sort_values(by='abs_importance', ascending=False).drop(columns='abs_importance').reset_index(drop=True)

    top_ratios = feature_importance_sorted.head(5)['feature'].tolist()
    def get_custom_bins(col):
        # Add a small random noise to avoid duplicate bin edges
        noise = np.random.normal(0, 1e-5, size=len(col))
        col_noisy = col + noise
        # Calculate percentiles to ensure unique bin edges
        bins = np.percentile(col_noisy, [0, 20, 40, 60, 80, 100])
        return bins

    df_copy = df.copy()
    df_copy['score'] = 0  # Initialize the score column
    df_copy['original_order'] = range(len(df_copy))  # Add original order


    for ratio in top_ratios:
        df_sort = df_copy.sort_values(by=ratio)
        custom_bins = get_custom_bins(df_sort[ratio])
        df_sort['status'] = pd.cut(df_sort[ratio], bins=custom_bins, labels=['very low', 'low', 'inline', 'high', 'very high'], include_lowest=True)
        df_copy['status'] = df_sort.set_index('original_order')['status']

        for i in range(len(df_copy)):
          status = df_copy.iloc[i]['status']
          preference = 'high' if df[df['is_holding'] == 1][ratio].median() >= df[df['is_holding'] == 0][ratio].median() else 'low'

          if preference == 'high':
              if status == 'very high':
                  df_copy.at[i, 'score'] += score_set[-1]
              elif status == 'high':
                  df_copy.at[i, 'score'] += score_set[-2]
              elif status == 'inline':
                  df_copy.at[i, 'score'] += score_set[-3]
              elif status == 'low':
                  df_copy.at[i, 'score'] += score_set[-4]
              elif status == 'very low':
                  df_copy.at[i, 'score'] += score_set[-5]
          else:
              if status == 'very high':
                  df_copy.at[i, 'score'] += score_set[0]
              elif status == 'high':
                  df_copy.at[i, 'score'] += score_set[1]
              elif status == 'inline':
                  df_copy.at[i, 'score'] += score_set[2]
              elif status == 'low':
                  df_copy.at[i, 'score'] += score_set[3]
              elif status == 'very low':
                  df_copy.at[i, 'score'] += score_set[4]
          # Rank the DataFrame with scores and sort it
    df_ranked = df_copy.sort_values(by='score', ascending=False)
    df_ranked['predicted_is_holding'] = df_ranked['score'].apply(lambda x: 1 if x > 0 else 0)

    return df_ranked

In [188]:
from statistics import median, mode

def generate_fund_sentence(df, company, top_ratios, top_n_company, predict_hold = False):
    likes = []
    dislikes = []
    comments = []
    if 'predicted_is_holding' not in df.columns:
        df['predicted_is_holding'] = 0  # Initialize the 'predicted_is_holding' column with 0

    if predict_hold == False:
      # comany score from lightfm top n predict hold
      sorted_scores = np.sort(df['score'].values)[::-1]
      threshold = sorted_scores[min(top_n_company, len(sorted_scores)-1)]

      score = df[df['company'] == company]['score'].values[0]
      if score >= threshold:
          df.loc[df['company'] == company, 'predicted_is_holding'] = 1
          comments.append(f"I like company {company}")
      else:
          df.loc[df['company'] == company, 'predicted_is_holding'] = 0
          comments.append(f"I dislike company {company}")

      non_holdings = df[df['predicted_is_holding'] == 0]
      holdings = df[df['predicted_is_holding'] == 1]

    else:
      if df[df['company'] == company]['predicted_is_holding'].values[0] == 1:
        comments.append(f"I like company {company}")
      else:
        comments.append(f"I dislike company {company}")

      non_holdings = df[df['predicted_is_holding'] == 0]
      holdings = df[df['predicted_is_holding'] == 1]

    non_holding = non_holdings[feature_cols]
    holding = holdings[feature_cols]

    if df[df['Ticker'] == company].empty:
        print(f'{company} has no data in holding history')
    else:
      for top_ratio in top_ratios:
          ratio_holding_median = median(holding[top_ratio].values)
          ratio_non_holding_median = median(non_holding[top_ratio].values)

          company_value = df[df['Ticker'] == company][top_ratio].values[0]
          overall_median = median(df[top_ratio].values)
          if ratio_holding_median >= ratio_non_holding_median:
              if company_value >= overall_median:
                  likes.append(f"because {top_ratio} is high")
          else:
              if company_value < overall_median:
                  likes.append(f"because {top_ratio} is low")

          if ratio_holding_median < ratio_non_holding_median:
              if company_value >= overall_median:
                  dislikes.append(f"because {top_ratio} is high")
          else:
              if company_value < overall_median:
                  dislikes.append(f"because {top_ratio} is low")

    return likes, dislikes, comments


# Preprocess combined_data

In [155]:
# Method 1
combined_data = pd.read_parquet(path + 'Data/combined_data.parquet')

# Standardize the ratios
ratio_columns = combined_data.columns.difference(drop_cols)
scaler = StandardScaler()
combined_data[ratio_columns] = scaler.fit_transform(combined_data[ratio_columns])

# Scale the standardized ratios to a specific range, e.g., -1 to 1
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
combined_data[ratio_columns] = min_max_scaler.fit_transform(combined_data[ratio_columns])

In [156]:
funds = combined_data.groupby('Type')['Fund'].unique()
funds

,Fund
Type,
dividend,"[WTMCX, GAINX, EDFAX, WCDGX, ZDIVX, FEQTX, FET..."
growth,"[MPGEX, MPGIX, PJFBX, PRGIX, TGTIX, CATEX, FOC..."
value,"[VUVLX, DPDEX, ACVVX, GOWLX, CAMAX, WSCVX, QRS..."


In [157]:
dividend_funds = funds['dividend'].tolist()
growth_funds = funds['growth'].tolist()
value_funds = funds['value'].tolist()

#Explain Value Fund

In [158]:
# only run 1 time
company_scores['Date'] = pd.to_datetime(company_scores['Date']).dt.strftime('%Y-%m-%d')
combined_data['Backtest Date'] = combined_data['Backtest Date'].dt.strftime('%Y-%m-%d')

In [ ]:
style = 'value'
top_n_ratios = 5
top_n_companies = 3
n = 5
score_set = [-8, -6, 0, 5, 10]
backtest_dates = combined_data[combined_data['Type'] == style]['Backtest Date'].unique()

for i in backtest_dates:
    scores = company_scores[company_scores['Date'] == i]

    selected_companies = scores['company'].tolist()[:top_n_companies] # top n companies
    # selected_companies = scores['company'].sample(n=3, random_state=42).tolist() # random 3 companies

    for company in selected_companies:
        print(f"At backtest date {i}:")
        print(f"Company {company}:")
        print("    method LightFM:")

        for fund in value_funds:
            fund_df = combined_data[(combined_data['Fund'] == fund) & (combined_data['Backtest Date'] == i)]
            # drop the duplication Tickers that shows in 2nd time
            fund_df = fund_df.drop_duplicates(subset='Ticker', keep='first')
            fund_df = pd.merge(scores, fund_df, left_on='company', right_on='Ticker')

            if fund_df.empty:
                continue

            # use actual is_holding to decide top ratios
            top_ratios = find_top_ratios(fund_df, top_n_ratios, feature_cols)
            fund_df = generate_scores(fund_df, top_ratios, score_set, feature_cols)
            # use predicted is_holding to explain
            if 'predicted_is_holding' in fund_df.columns:
              likes, dislikes, comments = generate_fund_sentence(fund_df, company, top_ratios, n, predict_hold = True)
            else:
              likes, dislikes, comments = generate_fund_sentence(fund_df, company, top_ratios, n, predict_hold = False)

            print(f"        {style} Fund {fund}:")
            for comment in comments:
                print(f"            {comment}")
                if comment.startswith("I like"):
                    for reason in likes:
                        print(f"            {reason}")
                else:
                    for reason in dislikes:
                        print(f"            {reason}")


At backtest date 2021-03-31:
Company SEER:
    method LightFM:
        value Fund VUVLX:
            I dislike company SEER
            because Return On Invested Capital is low
            because Operating Margin is low
            because Return on Assets is low
            because Return on Equity is low
At backtest date 2021-03-31:
Company BOLT:
    method LightFM:
        value Fund VUVLX:
            I dislike company BOLT
            because Return On Invested Capital is low
            because Operating Margin is low
            because Return on Assets is low
At backtest date 2021-03-31:
Company TGTX:
    method LightFM:
        value Fund VUVLX:
            I dislike company TGTX
            because Return On Invested Capital is low
            because Operating Margin is low
            because Return on Assets is low
            because Return on Equity is low
At backtest date 2021-09-30:
Company BILL:
    method LightFM:
        value Fund DPDEX:
            I dislike com